In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Промежуточный проект_e-learning/assessments.csv')
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Промежуточный проект_e-learning/courses.csv')
stud_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Промежуточный проект_e-learning/studentAssessment.csv')
stud_registration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-t-varenichenko-22/My_practice/Промежуточный проект_e-learning/studentRegistration.csv')


#### 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)

In [4]:
# 1.1 Посмотрим по каким предметам предполагалась сдача экзамена 
assessments_exam_11 = assessments.query("assessment_type == 'Exam'")[['code_presentation', 'id_assessment']]

# 1.2 Отберем только тех студентов, кто успешно сдал экзамен с первого раза
stud_ass_exam = stud_assessment.query('score >= 40 & is_banked == 0')
# 4392 rows 

# 1.4 Добавим к данным по оценкам студентов, успешно сдавших экзамен, информацию о семестре и id экзамена 
stud_101 = stud_ass_exam[['id_assessment', 'id_student']].merge(assessments_exam_11, on = 'id_assessment')

# 1.5 Сколько всего студентов успешно сдавали по 1 экзамену
one_exam = stud_101 \
                    .groupby('id_student', as_index = False) \
                    .agg({'id_assessment': 'count'}) \
                    .rename(columns = {'id_assessment' : 'exam_number'}) \
                    .query('exam_number == 1') \
                    .exam_number \
                    .sum()
one_exam
# # 3802

# 1.6 Студенты в каждом семестре, которые успешно сдали всего один экзамен
one_exam_10 = stud_101 \
                .groupby(['code_presentation','id_student'], as_index = False) \
                .agg({'id_assessment': 'count'}) \
                .rename(columns = {'id_assessment' : 'exam_number'}) \
                .query('exam_number == 1') \
                .pivot_table(index= 'code_presentation', values="exam_number", aggfunc=np.sum) \
                .reset_index()
one_exam_10

,code_presentation,exam_number
0,2013B,504
1,2013J,878
2,2014B,1131
3,2014J,1767


In [8]:
'''Студенты, которые сдавали 2 экзамена и успешно сдали только один'''

# Еще раз отберем только экзамены
assessments_exam_101 = assessments.query("assessment_type == 'Exam'")[['code_presentation', 'id_assessment', 'code_module']]

# Добавим к данным по оценкам студентов информацию о семестре и id экзамена
completion = stud_assessment.merge(assessments_exam_101, on = 'id_assessment')

completion['passed'] = (completion['score'] >= 40)*1

# Отберем нужные столбцы
df_11 = completion.groupby('id_student', as_index = False)\
                            .agg({'id_assessment': 'count',
                                  'passed': 'sum'})

df_11.query('id_assessment == 2 and passed == 1').id_student.count()


28

#### 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью. 
##### *Завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен    

In [4]:
# 0. Еще раз отберем только экзамены
assessments_exam_101 = assessments.query("assessment_type == 'Exam'")[['code_presentation', 'id_assessment', 'code_module']]

# 2.1. Добавим к данным по оценкам студентов информацию о семестре и id экзамена
completion = stud_assessment.merge(assessments_exam_101, on = 'id_assessment')
# 4959 rows 

# 2.2.Есть ли нули в получившемся датафрейме
completion.isna().sum()
# 0

# 2.3. Посмотрим общее количество попыток сдать экзамен
total_attempts = completion \
                                .groupby(['code_presentation', 'id_assessment', 'code_module'], as_index = False) \
                                .agg({'score' : 'count'}) \
                                .rename(columns = {'score' : 'total_attempts'}) \
                                .sort_values('code_presentation')

# 2.4. Посмотрим общее количество успешных попыток
successfull_attempts = completion.query('score >= 40') \
                                 .groupby(['code_presentation', 'id_assessment'], as_index = False) \
                                 .agg({'score' : 'count'}) \
                                 .rename(columns = {'score' : 'successfull_attempts'}) \
                                 .sort_values('code_presentation')

# 2.5. Соединим информацию
completion_1 = total_attempts.merge(successfull_attempts[['successfull_attempts', 'id_assessment']], on = 'id_assessment')

# 2.6.  Определим завершаемость курсов в семестре
completion_1['completion_rate'] = completion_1.successfull_attempts.div(completion_1.total_attempts).mul(100).round(2)
completion_1


,code_presentation,id_assessment,code_module,total_attempts,successfull_attempts,completion_rate
0,2013B,25340,DDD,602,504,83.72
1,2013J,25354,DDD,968,878,90.70
2,2014B,24290,CCC,747,664,88.89
3,2014B,25361,DDD,524,485,92.56
4,2014J,24299,CCC,1168,1019,87.24
5,2014J,25368,DDD,950,842,88.63


##### Таким образом, за все время наиболее сложным был экзамен 25340 в семестре 2013B. Наиболее легким (завршаемость 92.56%) - 25361 в семестре 2014B. 
##### В этом же году студенты сдавали предмет с кодом ССС и по сравнению с предметом DDD он оказался сложнее: завершаемость в обоих семестрах была ниже.

#### 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [7]:
# 3.1 Добавим к оценкам информацию о длине семестра
assessments_date = assessments.query("assessment_type == 'Exam'")[['code_module','code_presentation', 'id_assessment']] \
                              .merge(courses, on = ['code_module', 'code_presentation'])

# 3.2. Добавим к данным по оценкам студентов, информацию о семестре и id экзамена 
stud_date = stud_assessment.merge(assessments_date, on = 'id_assessment').drop('is_banked', axis = 1)

# 3.3 Вычислим длину сессии
stud_date['session_length'] = stud_date.module_presentation_length - stud_date.date_submitted.min()

# 3.4 Найдем день проведения экзамена каждого студента
stud_date['exam_date'] = stud_date.module_presentation_length - stud_date.date_submitted

# 3.5 Посмотрим все ли экзамены были сданы вовремя
stud_date['late_submition'] = stud_date.date_submitted > stud_date.module_presentation_length

# 3.6 Средняя длина сессии для успешных экзаменов в рамках основной сессии
success_submition_dates = stud_date \
                            .query('score >= 40 & late_submition == False') \
                            .groupby(['code_presentation', 'code_module', 'id_assessment', 'session_length'], as_index = False) \
                            .agg({'exam_date':'mean', 'date_submitted': 'mean'}) \
                            .rename(columns ={'exam_date': 'avg_succ_session', 'date_submitted': 'mean_submit_date' }) \
                            .sort_values('code_presentation') \
                            .round(0) \
                            .reset_index()
success_submition_dates

,index,code_presentation,code_module,id_assessment,session_length,avg_succ_session,mean_submit_date
0,0,2013B,DDD,25340,11,10.0,230.0
1,1,2013J,DDD,25354,32,21.0,240.0
2,2,2014B,CCC,24290,12,9.0,232.0
3,3,2014B,DDD,25361,12,6.0,235.0
4,4,2014J,CCC,24299,40,25.0,244.0
5,5,2014J,DDD,25368,33,19.0,243.0


#### 4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3)

In [8]:
# 4.0 Проверим есть ли пропущенные значения
stud_registration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [9]:
'''В описании датасета указано: если учащийся не отправляет работу на оценку, результат не записывается в таблицу.
Удалим данные по тем студентам, которые закончили курс (date_unregistration == NaN), но у которых нет записи в таблице оценок и которые
не имеют даты регистрации на него (date_registration == NaN). Такие данные будем рассматривать как ошибку ввода. 
Записи, где есть дата отзыва регистрации, но нет информации о записи на курс оставим'''

# 4.1 Проверим сколько студентов зарегистрированы на курс, но не имеют оценок
list = stud_assessment.id_student.unique().tolist()
check = stud_registration.query('id_student not in list')
# В таблице оценок студентов отсутствуют записи 5847 учащихся

# Проверим сколько из этих студентов не имеют записи об отмене регистрации
check.isnull().sum()

code_module               0
code_presentation         0
id_student                0
date_registration        36
date_unregistration    1253
dtype: int64

In [37]:
''' Будем использовать для подсчета популярности курсов'''
# 4.2 Удалим записи тех, кто не имеет информации о регистарции (поступлении и отчислении)
registration = stud_registration.dropna(thresh=4)

In [38]:
# 4.3.Сначала посмотрим на студентов, окончивших курc и удалим колонку date_unregistration для удобства
finished_courses = registration.query("date_unregistration == 'NaN'").drop('date_unregistration',axis=1 )
# 21515 rows

In [41]:
# Проверим, нет ли повторений регистраций на курсах
finished_courses.id_student.nunique()

21185

In [44]:
# 4.3 Проведем первичную группировку 
by_presentation = finished_courses \
                                        .groupby(['code_presentation', 'code_module'], as_index = False) \
                                        .agg({'date_registration': 'mean', 'id_student': 'nunique'}) \
                                        .rename(columns = {'id_student' : 'reg_num', 'date_registration' : 'mean_reg_date'}) \
                                        .round(0)

# Создадим вспомогательную колонку total_reg
by_code = by_presentation \
                            .groupby('code_presentation', as_index = False) \
                            .agg({'reg_num': 'sum'}) \
                            .round(0) \
                            .rename(columns = {'reg_num' : 'total_reg_by_semestr'})

# Добавим total_reg к таблице регистраций по семестрам
by_code_presentation = by_presentation.merge(by_code[['code_presentation', 'total_reg_by_semestr']], on = 'code_presentation')

# 4.4 И помотрим процентное соотношение регистраций в семестре
by_code_presentation['registration_rate'] = by_code_presentation.reg_num.div(by_code_presentation.total_reg_by_semestr).mul(100).round(1)

# Удалим вспомогательную колонку
by_code_presentation = by_code_presentation.drop('total_reg_by_semestr', 1)
by_code_presentation.head(5)

,code_presentation,code_module,mean_reg_date,reg_num,registration_rate
0,2013B,BBB,-56.0,1261,37.8
1,2013B,DDD,-59.0,870,26.1
2,2013B,FFF,-58.0,1202,36.1
3,2013J,AAA,-77.0,323,5.0
4,2013J,BBB,-64.0,1590,24.6


##### ТОП-3 курса по общему количеству зарегистрированных среди всех курсов и семестров

In [13]:
top_finished_courses = by_code_presentation \
                                .nlargest(3, 'reg_num')[['code_module', 'code_presentation', 'reg_num', 'registration_rate']] \
                                .reset_index(drop=True)
top_finished_courses

,code_module,code_presentation,reg_num,registration_rate
0,FFF,2013J,1606,24.8
1,BBB,2013J,1590,24.6
2,BBB,2014J,1556,20.7


##### ТОП - 3 наиболее популярных курсов за все время наблюдений

In [45]:
finished_courses \
                .groupby(['code_module'], as_index = False) \
                .agg({'date_registration': 'mean', 'id_student': 'nunique'}) \
                .rename(columns = {'id_student' : 'reg_num', 'date_registration' : 'mean_reg_date'}) \
                .round(0) \
                .sort_values('reg_num', ascending = False) \
                .head(3)

,code_module,mean_reg_date,reg_num
1,BBB,-64.0,5511
5,FFF,-66.0,5355
3,DDD,-72.0,4007


##### Наиболее популярные курсы в каждом семестре

In [46]:
by_code_presentation \
                    .groupby('code_presentation', as_index = False) \
                    .apply(lambda x : x.sort_values(by = 'reg_num', ascending = False) \
                    .head(1) \
                    .reset_index()) [['code_module', 'code_presentation', 'reg_num', 'registration_rate']]


,,code_module,code_presentation,reg_num,registration_rate
0,0,BBB,2013B,1261,37.8
1,0,FFF,2013J,1606,24.8
2,0,BBB,2014B,1124,21.6
3,0,BBB,2014J,1556,20.7


#### Отток с курсов

In [48]:
# Посмотрим на курсы оттоком
withdraw_courses = registration.query("date_unregistration != 'NaN'")

In [49]:
withdraw_courses.isnull().sum()

code_module             0
code_presentation       0
id_student              0
date_registration      39
date_unregistration     0
dtype: int64

In [50]:
# Посмотрим отдельно на курсы, где нет дат регистрации
withdraw_courses_date = withdraw_courses.query("date_registration != 'NaN'")

# Получим медианную дату регистрации
median = withdraw_courses_date.date_registration.median()
median

-67.0

In [52]:
'''4.9 Допущение: для выбывших студентов не будем ставить дату регистрации после даты отмены регистрации. 
Вычтем медианное значение даты регистрации из даты отмены регистрации, чтобы заполнить их'''
  
withdraw_courses.loc[withdraw_courses.date_registration.isna(), 'date_registration'] = withdraw_courses['date_unregistration'] + median

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [54]:
# 4.10 Посмотрим сколько студентов отчислились после начала модуля
withdraw_courses['after_semester_start'] = withdraw_courses.date_unregistration >= 0 

withdraw_courses.after_semester_start.value_counts(normalize = True)
# '''Больше 70% студентов отчисляются после начала семестра'''

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


True     0.734114
False    0.265886
Name: after_semester_start, dtype: float64

In [57]:
# Посмотрим сколько дней проходит с момента регистрации до отчисления
withdraw_courses['days_before_unregistration'] = abs(withdraw_courses.date_registration - withdraw_courses.date_unregistration )

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
# 4.11 Посмотрим как разделяются стюденты по времени отзыва регистрации на курс
bins = [-366, -180, -90, -30, 0, 30, 90, 180, 269]
labels = ['more than 6 months',
          '3 to 6 months prior',
          '1 to 3 months prior', 
          'month prior', 
          'within start month', 
          '1 to 3 months', 
          '3 to 6 months', 
          'after longest module end']

withdraw_courses['unregistration_time'] = pd.cut(withdraw_courses.date_unregistration, bins, labels = labels )

# Больше половины студентов отчисляется с курса за месяц до его начала или в первые 3-6 месяцев
withdraw_courses.groupby(['unregistration_time'], as_index = False).id_student.agg('count')

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,unregistration_time,id_student
0,more than 6 months,29
1,3 to 6 months prior,295
2,1 to 3 months prior,834
3,month prior,1939
4,within start month,2030
5,1 to 3 months,1904
6,3 to 6 months,2287
7,after longest module end,753


##### ТОП - 3 модулей по оттоку студентов за все время

In [58]:
withdraw_courses \
                .groupby('code_module', as_index = False) \
                .agg({'id_student': 'nunique', 'days_before_unregistration': 'mean'} ) \
                .rename(columns = {'id_student' : 'unreg_number', 'days_before_unregistration': 'mean_days_before_unregistration'}) \
                .round(0) \
                .sort_values('unreg_number', ascending = False) \
                .head(3)

,code_module,unreg_number,mean_days_before_unregistration
1,BBB,2314,112.0
5,FFF,2249,127.0
3,DDD,2065,141.0


In [61]:
# Вернемся к курсам с наибольшим оттоком студентов за все время и посмотрим в какие даты происходит больше всего отчислений
wd = ['FFF', 'DDD', 'BBB']
top_withdraw_courses = withdraw_courses.query('code_module in @wd').reset_index()
top_withdraw_courses.head()

,index,code_module,code_presentation,id_student,date_registration,date_unregistration,after_semester_start,unregistration_time,days_before_unregistration
0,750,BBB,2013B,27891,-58.0,153.0,True,3 to 6 months,211.0
1,753,BBB,2013B,33666,-23.0,139.0,True,3 to 6 months,162.0
2,756,BBB,2013B,34431,-24.0,27.0,True,within start month,51.0
3,757,BBB,2013B,34694,-55.0,-54.0,False,1 to 3 months prior,1.0
4,758,BBB,2013B,34756,-43.0,59.0,True,1 to 3 months,102.0


In [62]:
# Для модуля BBB большинство студентов отчисляется за месяц до начала семестра, 
# при поступлении на модули DDD и FFF большинство отчилсяется в период 3-6 месяца обучения

top_withdraw_courses.pivot_table(index = ['code_module'], columns = 'unregistration_time' , values = 'id_student', aggfunc='nunique' )

unregistration_time,more than 6 months,3 to 6 months prior,1 to 3 months prior,month prior,within start month,1 to 3 months,3 to 6 months,after longest module end
code_module,,,,,,,,
BBB,4,85,249,642,497,326,404,143
DDD,9,53,163,367,404,421,553,222
FFF,6,66,178,436,488,464,547,152


In [63]:
# Проведем первичную группировку
withdraw_by_presentation = withdraw_courses \
                                        .groupby(['code_presentation', 'code_module'], as_index = False) \
                                        .agg({'date_registration': 'mean', 'id_student': 'nunique'}) \
                                        .rename(columns = {'id_student' : 'unreg_number', 'date_unregistration' : 'mean_unreg_date'}) \
                                        .round(0)

# Создадим вспомогательную колонку total_unreg
withdraw_by_code = withdraw_by_presentation \
                                            .groupby('code_presentation', as_index = False) \
                                            .agg({'unreg_number': 'sum'}) \
                                            .round(0) \
                                            .rename(columns = {'unreg_number' : 'total_unreg_by_semestr'})

# Добавим total_unreg к таблице отчислений по семестрам
withdraw_by_code_presentation = withdraw_by_presentation.merge(withdraw_by_code[['code_presentation', 'total_unreg_by_semestr']], on = 'code_presentation')

# И помотрим процентное соотношение регистраций в семестре
withdraw_by_code_presentation['unregistration_rate'] = withdraw_by_code_presentation.unreg_number.div(withdraw_by_code_presentation.total_unreg_by_semestr).mul(100).round(1)

# Удалим вспомогательную колонку
withdraw_by_code_presentation = withdraw_by_code_presentation.drop('total_unreg_by_semestr', 1)
withdraw_by_code_presentation.head(5)

,code_presentation,code_module,date_registration,unreg_number,unregistration_rate
0,2013B,BBB,-67.0,505,37.5
1,2013B,DDD,-64.0,431,32.0
2,2013B,FFF,-66.0,411,30.5
3,2013J,AAA,-83.0,60,2.5
4,2013J,BBB,-83.0,647,27.2


##### ТОП-3 курса по общему количеству отозвавших заявки в разрезе семестров

In [64]:
top_unfinished_courses = withdraw_by_code_presentation \
                                .nlargest(3, 'unreg_number')[['code_module', 'code_presentation', 'unreg_number', 'unregistration_rate']] \
                                .reset_index(drop=True)
top_unfinished_courses

,code_module,code_presentation,unreg_number,unregistration_rate
0,CCC,2014J,1049,28.1
1,CCC,2014B,898,34.4
2,FFF,2014J,831,22.2


#### Курсы в семестре с наибольшим оттоком

In [65]:
withdraw_by_code_presentation \
    .groupby('code_presentation', as_index = False) \
    .apply(lambda x : x.sort_values(by = 'unreg_number', ascending = False) \
    .head(1) \
    .reset_index()) [['code_module', 'code_presentation', 'unreg_number', 'unregistration_rate']]

,,code_module,code_presentation,unreg_number,unregistration_rate
0,0,BBB,2013B,505,37.5
1,0,DDD,2013J,684,28.8
2,0,CCC,2014B,898,34.4
3,0,CCC,2014J,1049,28.1


#### 5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов

In [33]:
# 5.1 Соединим нужные таблицы
completion = assessments.query("assessment_type == 'Exam'")[['code_presentation', 'id_assessment', 'code_module']] \
                        .merge(courses, on = ['code_module', 'code_presentation']) \
                        .merge(stud_assessment, on = 'id_assessment') \
                        .drop('is_banked', axis = 1)

# 5.2 Добавим длину сессии
completion['session_length'] = completion.module_presentation_length - completion.date_submitted.min()

# 5.3 Добавим день проведения экзамена каждого студента
completion['exam_date'] = completion.module_presentation_length - completion.date_submitted

# 5.4 Заменим названия семестра
comp = completion.replace({'code_presentation' : { '2013B' : 1, '2013J' : 2, '2014B' : 3, '2014J': 4}})

# 5.5 Посмотрим, когда студент сдал свой первый экзамен, то есть определим в какую когорту он попадет 
student_data_1 = pd.DataFrame(comp.groupby('id_student')['code_presentation'].min())
student_data_1.columns = ['student_first_semestr']

# 5.6 Добавим семестр сдачи первого экзамена
df = pd.merge(comp, student_data_1.reset_index(), on='id_student')

# 5.7 определим сдал экзамен или нет
df['passed'] = (df['score'] >= 40)*1

In [34]:
# 5.8 создадим индекс когорты
df['CohortIndex'] = df.code_presentation - df.student_first_semestr + 1


''' 5.9 Создаем когорты'''
def generate_cohorts(dataset, metric):
    cohorts = dataset.groupby(['student_first_semestr','CohortIndex']).agg({
        'id_student': pd.Series.nunique,
        'id_assessment' : pd.Series.count, 
        'passed': pd.Series.sum,
        'date_submitted': 'mean'
    })
    cohorts.columns = ['number_of_students','number_of_exams','total_passed_exam', 'avg_exam_time']
    cohorts = cohorts[metric].unstack(1)
    
    return cohorts

'''5.10 Завершаемость курсов'''

def ratio(dataset):
    cohort_data_2 = df.groupby(['student_first_semestr','CohortIndex']).agg({'id_assessment' : pd.Series.count,                                                                         'id_assessment' : pd.Series.count, 
                                                                         'passed': np.sum}) \
                                                                        .reset_index()
                                                                    
    cohort_data_2.rename(columns={'id_assessment': 'total_exams'}, inplace=True)
    # переиндексируем DataFrame
    cohort_data_2.set_index(['CohortIndex','student_first_semestr'], inplace=True)

    # создадим ряд содержаший размер каждой когорты student_first_semestr
    cohort_group_size = cohort_data_2['total_exams'].groupby(level=1).first()

    pass_rate = cohort_data_2['passed'].unstack(0).divide(cohort_group_size, axis=0)
    return pass_rate


In [35]:
generate_cohorts(df, metric='avg_exam_time')

CohortIndex,1,2,3,4
student_first_semestr,,,,
1,230.179402,NaN,231.622222,244.625
2,239.505165,231.297872,244.191919,NaN
3,233.108567,244.507463,NaN,NaN
4,243.615226,NaN,NaN,NaN


In [36]:
ratio(df)

CohortIndex,1,2,3,4
student_first_semestr,,,,
1,0.837209,NaN,0.063123,0.009967
2,0.907025,0.044421,0.099174,NaN
3,0.905852,0.047498,NaN,NaN
4,0.876029,NaN,NaN,NaN


#### 6. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен.

In [15]:
# 6.1 Соединим нужные таблицы
completion = assessments.query("assessment_type == 'Exam'")[['code_presentation', 'id_assessment', 'code_module']] \
                        .merge(courses, on = ['code_module', 'code_presentation']) \
                        .merge(stud_assessment, on = 'id_assessment') \
                        .drop('is_banked', axis = 1)

# 6.2 Добавим длину сессии
completion['session_length'] = completion.module_presentation_length - completion.date_submitted.min()

# 6.3 Добавим день проведения экзамена каждого студента
completion['exam_date'] = completion.module_presentation_length - completion.date_submitted

# 6.4 определим сдал экзамен или нет
completion['passed'] = (completion['score'] >= 40)*1

# 6.5 Отберем нужные столбцы
df_1 = completion.groupby('id_student', as_index = False)\
                            .agg({'date_submitted':'mean',
                                  'score': 'mean',
                                  'id_assessment': 'count',
                                  'passed': 'sum'})

# 6.6 Найдем завершаемость курсов для каждого студента
df_1['comp_rate'] = df_1.passed / df_1.id_assessment

# 6.7 Соберем табличку для RFM-анализа
for_RFM_analysis = df_1[['id_student', 'date_submitted', 'comp_rate', 'score']] \
                        .rename(columns = {'date_submitted': 'R',
                                           'comp_rate' : 'F',
                                           'score' : 'M'})


In [32]:
# 6.8 Для деления на кластеры по recency будем использовать процентили   
for_RFM_analysis.R.describe()

# 6.9 Создадим кластер r_quartile 
for_RFM_analysis['r_cluster'] = pd.qcut(for_RFM_analysis['R'], 4, ['1','2','3','4'])


# 6.10 Так как разных значений по полю "завершенность курсов" (F) всего 3, разделим студентов на три кластера   
    # 3 - не сдал ни один экзамен
    # 2 - сдал один экзамен из 2х
    # 1 - сдал два экзамена
bins = [0, 0.49, 0.7, 1]
labels = ['3', '2', '1']
for_RFM_analysis['f_cluster'] = pd.cut(for_RFM_analysis['F'], bins, labels = labels )
for_RFM_analysis['f_cluster'] = for_RFM_analysis['f_cluster'].cat.add_categories('nan').fillna('3')

# 6.11 По среднему баллу студентов разделим на кластеры:
#  - не сдали экзамен: М < 40
#  - сдали экзамен,с шагом в 20 баллов:
#         40-69 баллов - тройка, кластер 4
#         70-89 баллов - четверка, кластер 2
#         90-100 - пятерка, кластер 1

bins = [0, 39, 69, 89, 100]
labels = ['4','3','2', '1']

for_RFM_analysis['m_cluster'] = pd.cut(for_RFM_analysis['M'], bins, labels = labels )

# Среди оценок так же есть нулевые значения
for_RFM_analysis['m_cluster'] = for_RFM_analysis['m_cluster'].cat.add_categories('nan').fillna('4')

In [33]:
# 6.12 Объединим все кластеры в один столбец-рейтинг для сегментации групп студентов
for_RFM_analysis['RFM_Score'] = for_RFM_analysis.r_cluster.astype(str) \
                                + for_RFM_analysis.f_cluster.astype(str) \
                                + for_RFM_analysis.m_cluster.astype(str)
for_RFM_analysis.head()

,id_student,R,F,M,r_cluster,f_cluster,m_cluster,RFM_Score
0,23698,243.0,1.0,80.0,3,1,2,312
1,24213,236.0,1.0,58.0,2,1,3,213
2,27116,243.0,1.0,96.0,3,1,1,311
3,28046,237.0,1.0,40.0,2,1,3,213
4,28787,243.0,1.0,44.0,3,1,3,313


In [34]:
'''6.13 Студенты с самым низким средним временем сдачи (ранняя сдача экзамена), самой высокой завершаемостью 
и высоким средним баллом считаются лучшими студентами '''
for_RFM_analysis[for_RFM_analysis['RFM_Score']=='111'].id_student.count()

207

In [35]:
# Посмотрим как распределились студенты по кластерам
for_RFM_analysis.RFM_Score.value_counts()

113    667
313    606
213    432
112    420
312    368
413    357
212    308
412    239
411    226
111    207
134    179
334    150
311    140
434    117
211     99
234     90
223     10
123      8
224      5
323      3
424      1
124      1
Name: RFM_Score, dtype: int64

In [36]:
# Посмотрим на студентов наиболее многочисленной группы
for_RFM_analysis[for_RFM_analysis['RFM_Score']=='113']

,id_student,R,F,M,r_cluster,f_cluster,m_cluster,RFM_Score
21,40604,234.0,1.0,66.0,1,1,3,113
22,42638,230.0,1.0,50.0,1,1,3,113
24,45664,230.0,1.0,40.0,1,1,3,113
40,57340,230.0,1.0,68.0,1,1,3,113
44,60135,229.0,1.0,40.0,1,1,3,113
...,...,...,...,...,...,...,...,...
4601,2669736,234.0,1.0,64.0,1,1,3,113
4608,2677955,230.0,1.0,69.0,1,1,3,113
4618,2688692,234.0,1.0,51.0,1,1,3,113
4619,2689536,229.0,1.0,58.0,1,1,3,113


Так, среди студентов преобладают те, кто сдал экзамен в ранние сроки, но с низкой средней оценкой